In [34]:
# This notebook exports a Catchment_attributes.csv file for Basin Delineation B

In [35]:
import pandas as pds
import numpy as np
from pathlib import Path
import geopandas as gpd
import geopandas as gpds

In [36]:
# Define paths to files
gauges_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\gauges_with_splitted_included.shp') 
wsheds_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_B.shp')

soil_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\soil_attrs_basins_B.csv")
NI_geo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\NI_geo_attrs_basins_B.csv")
topo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\topo_attrs_Basins_B.csv")
clim_ind_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Clim_ind_1989_2009_ravII.csv")
clim_ind_path_era5L = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Clim_ind_1989_2009.csv")

# Read gauges shapefile
gauges = gpds.read_file(gauges_path)
gauges = gauges.set_index('id')
cols = ['st_numer', 'st_nafn', 'vhm_numer', 'vatnsfall', 'gagnaeigan', 'stada', 'upphaf_rek','dags_aflag', 'elevation']
gauges_reduced = gauges[cols]
gauges_reduced.columns = ['V_no','name','VHM_no','river','data_owner','status','upphaf_rek','dags_aflag', 'elevation']

# Read watershed attributes (calculated in QGIS)
wsheds_attrs = gpds.read_file(wsheds_path)
wsheds_attrs.index = wsheds_attrs['id']
wsheds_attrs = wsheds_attrs.drop(columns=['id'])
wsheds_attrs['lat'] = gauges.geometry.y
wsheds_attrs['lon'] = gauges.geometry.x
# Drop columns from wsheds_attrs that have no values
wsheds_attrs.dropna(axis=1, how='all', inplace=True)
# Extract only the relevant columns from wsheds_attrs
wsheds_attrs = wsheds_attrs[['gc_dom', 'strm_dens', 'mvert_dist', 'gc_ss_fra', 'area_calc', 'gc_wb_fra', 'litho_vapy', 'gc_py_fra', 'litho_dom', 'gc_ig_fra', 'elon_ratio', 'lon', 'litho_pb', 'gc_sm_fra', 'lc_dom', 'mvert_ang', 'gc_vb_fra', 'gc_mt_fra', 'litho_vbsr', 'gc_pi_fra', 'gc_va_fra', 'gc_su_fra', 'litho_pavr', 'litho_vbpy', 'gc_pb_fra', 'litho_vb', 'lat', 'gc_sc_fra', 'gc_pa_fra','lai_max', 'lai_diff', 'ndvi_max', 'ndvi_min','gvf_max', 'gvf_diff','geometry']]

# Read corine files
corine_path = Path(r'C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\corine\corine_watersheds_fractions_python\Basins_B\corine_2018.csv')
corine_attrs = pds.read_csv(corine_path)
corine_attrs.set_index('Watershed ID',inplace=True)
corine_attrs = corine_attrs[['agr_fra','bare_fra','forest_fra','glac_fra','lake_fra','urban_fra','scrub_fra','wetl_fra']]

# Read the soil attributes where depth to bedrock is calculated by excluding the glaciated regions
soil_indices = pds.read_csv(soil_path)
soil_indices = soil_indices.set_index('Unnamed: 0')
soil_indices.index = soil_indices.index.rename('id')

NI_geo_attrs = pds.read_csv(NI_geo_attrs_path)
NI_geo_attrs = NI_geo_attrs.set_index('ID')
NI_geo_attrs.index = NI_geo_attrs.index.rename('id')

# Topographical attributes
topo_attrs = pds.read_csv(topo_attrs_path,sep=';')
topo_attrs = topo_attrs.set_index('ID')

# Read clim indices with rav-ii data
era5_data = pds.read_csv(clim_ind_path,sep=';') 
era5_data.index=era5_data['ID']
era5_data.index = era5_data.index.rename('id')

# Read clim indices with ERA-Land data
era5_data_real = pds.read_csv(clim_ind_path_era5L,sep=';') 
era5_data_real.index=era5_data_real['ID']
era5_data_real.index = era5_data_real.index.rename('id')
newcols = [x+'_ERA5L' for x in era5_data_real.columns]
era5_data_real.columns = newcols

In [37]:
# Base DataFrame
merged_df = wsheds_attrs.sort_index()

# List of DataFrames to merge using an outer join initially
dfs_to_merge_outer = [
    soil_indices, NI_geo_attrs, topo_attrs, corine_attrs, era5_data, era5_data_real
]

# Merge with outer join
for df in dfs_to_merge_outer:
    merged_df = merged_df.merge(df.sort_index(), left_index=True, right_index=True, how='outer')

# Update geometry from gauges if necessary
merged_df['geometry'] = gauges.geometry

# Check the final type (should be a DataFrame)
type(merged_df)

geopandas.geodataframe.GeoDataFrame

In [31]:
# We skip the "splitted" watersheds (affected by diversions)
merged_df = merged_df[:-4]

In [32]:
attrs_to_export  =  ['area_calc',
                     'elev_mean',
                     'elev_med',
                     'elev_ran',
                     'elev_std',
                     'slope_mean', 'mvert_dist','mvert_ang',
                     'asp_mean','elon_ratio','strm_dens',
                     
                     'p_mean',
                     'pet_mean',
                     'aridity',
                     'p_seasonality',
                     'frac_snow',
                     'high_prec_freq',
                     'high_prec_dur',
                     'high_prec_timing',
                     'low_prec_freq',
                     'low_prec_dur',
                     'low_prec_timing',
                     
                     'p_mean_ERA5L',
                     'pet_mean_ERA5L',
                     'aridity_ERA5L',
                     'p_seasonality_ERA5L',
                     'frac_snow_ERA5L',
                     'high_prec_freq_ERA5L',
                     'high_prec_dur_ERA5L',
                     'high_prec_timing_ERA5L',
                     'low_prec_freq_ERA5L',
                     'low_prec_dur_ERA5L',
                     'low_prec_timing_ERA5L',                    
                     
                     'lc_dom','agr_fra','bare_fra', 'forest_fra', 'glac_fra',
                     'lake_fra','urban_fra','scrub_fra','wetl_fra',
                     
                     'lai_max',
                     'lai_diff',
                     'ndvi_max',
                     'ndvi_min',
                     'gvf_max',
                     'gvf_diff',
                     
                     'bedrk_dep','root_dep','soil_poros','soil_tawc',
                     'sand_fra','silt_fra','clay_fra','grav_fra','oc_fra',
                     
                     'gc_dom','gc_pa_fra','gc_pb_fra', 'gc_va_fra','gc_vb_fra',
                      'litho_dom','litho_pavr','litho_pb','litho_vapy','litho_vbsr','litho_vbpy', 'litho_vb',
                     
                     'g_dom_NI',
                     'g621_fra',
                     'g701_fra',
                     'g743_fra',
                     'g746_fra',
                     'gbinn_fra',
                     'ggnew_fra',
                     'ggold_fra',
                     'ghraun_fra',
                     'gbnew_fra',
                     'gbold_fra',
                     'gmob_fra',
                     'gsgos_fra',
                     'gsinn_fra',
                     'gsn_fra',
                     'gsnew_fra',
                     'gsold_fra']

# Define the dictionary for column renaming
rename_dict = {
    'p_seasonality': 'p_season',
    'p_seasonality_ERA5L': 'p_season_ERA5L',
    'high_prec_freq': 'high_prec_fr',
    'high_prec_freq_ERA5L': 'high_prec_fr_ERA5L',
    'low_prec_freq': 'lo_prec_fr',
    'low_prec_freq_ERA5L': 'lo_prec_fr_ERA5L',
    'high_prec_dur': 'high_prec_du',
    'low_prec_dur': 'low_prec_du',
    'high_prec_dur_ERA5L': 'high_prec_du_ERA5L',
    'low_prec_dur_ERA5L': 'low_prec_du_ERA5L',
    'litho_vapy': 'gc_23_vapy',
    'litho_dom': 'gc_23_dom',
    'litho_pavr': 'gc_23_pavr',
    'litho_pb': 'gc_23_pb',
    'litho_vbsr': 'gc_23_vbsr',
    'litho_vbpy': 'gc_23_vbpy',
    'litho_vb': 'gc_23_vb',
    'pet_mean': 'ref_et_mean'
}
# Update your attrs_to_export list to reflect these new names
attrs_to_export_updated = [rename_dict.get(attr, attr) for attr in attrs_to_export]
# attrs_to_export_updated = attrs_to_export_updated + ['q_mean', 'runoff_ratio', 'hfd_mean', 'slope_fdc', 'baseflow_index_ladson', 'stream_elas', 'high_q_freq', 'high_q_dur', 'Q95', 'low_q_freq', 'low_q_dur', 'Q5']

# Now, rename the columns in the DataFrame
merged_dff_renamed = merged_df.rename(columns=rename_dict)
merged_dff_renamed.index = merged_dff_renamed.index.rename('id')
# Finally, export the DataFrame with updated column names to CSV
attributes_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Catchment_attributes.csv")
merged_dff_renamed[attrs_to_export_updated].round(3).to_csv(attributes_path,sep=';')